<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [16]:
import pandas as pd
import transformers
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import notebook
import torch
import numpy as np

In [17]:
RANDOM_STATE = 42

## Подготовка

#### Загрузка данных

In [18]:
try:
    comments = pd.read_csv('../datasets/toxic_comments.csv')
except FileNotFoundError:
    comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

comments = comments.rename(columns={comments.columns[0]: 'index'})
comments = comments.set_index('index')

In [19]:
comments

,text,toxic
index,,
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0


### Предобработка

#### Токенизация

In [20]:
labels = torch.tensor(comments['toxic'].values)

In [21]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

encoded_data = tokenizer(
    comments['text'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors="pt"
)

padded = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']

In [22]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
padded = padded.to(device)
attention_mask = attention_mask.to(device)


with torch.no_grad():
    outputs = model(input_ids=padded, attention_mask=attention_mask)
    features = outputs.last_hidden_state[:, 0, :]

## Обучение

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(
    features.cpu().numpy(), labels.cpu().numpy(), test_size=0.2, random_state=RANDOM_STATE
)

In [ ]:
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(train_features, train_labels)

f1 = f1_score(test_labels, clf.predict(test_features))
f1

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны